# Reranking for diversity

Prepare the dataset as introduced before:

In [1]:
import rsdiv as rs

loader = rs.MovieLens1MDownLoader()
ratings = loader.read_ratings()
items = loader.read_items()

Not only for categorical labels, but **rsdiv** also supports embedding for items. 

For example, but the pre-trained 300-dim embedding based on `wiki_en` by `fastText` can also be simply imported as:

In [2]:
emb = rs.FastTextEmbedder()
emb.embedding_list(['Comedy', 'Romance'])

array([-0.02061814,  0.06264187,  0.00729847, -0.04322025,  0.04619966,
       -0.02074092, -0.05985767,  0.0153491 , -0.04587296,  0.04358115,
        0.0363803 , -0.03621935,  0.0572546 , -0.0684942 ,  0.05058537,
       -0.00981992, -0.06256518, -0.01003897, -0.11075602,  0.05155418,
        0.07386651,  0.09843259, -0.05135573, -0.04916158,  0.04005588,
       -0.01796244,  0.03565527,  0.00790671, -0.01076034,  0.15225074,
        0.03511112, -0.02062796, -0.027656  ,  0.01874454,  0.02118531,
       -0.0405189 , -0.08307864, -0.02818003, -0.02225998,  0.07319436,
       -0.00295853, -0.02865792,  0.02169566, -0.02701117, -0.12821653,
       -0.0479857 , -0.00673807,  0.00793977, -0.02369628, -0.03261875,
       -0.04239529,  0.10760261, -0.0519722 ,  0.00412771, -0.02244868,
        0.03178337, -0.01188657, -0.0328042 ,  0.01090564, -0.04114595,
       -0.0477084 , -0.04346408, -0.03280935,  0.15318173, -0.04905943,
        0.03020229,  0.01592956, -0.0391795 , -0.04651821,  0.07

**rsdiv** supports various kinds of diversifying algorithms:

- [Maximal Marginal Relevance](https://www.cs.cmu.edu/~jgc/publication/The_Use_MMR_Diversity_Based_LTMIR_1998.pdf), MMR diversify algorithm:

In [3]:
div = rs.MaximalMarginalRelevance(lbd=0.3)

- Modified Gram-Schmidt, MGS diversify algorithm, also known as SSD([Sliding Spectrum Decomposition](https://arxiv.org/pdf/2107.05204.pdf)):

In [4]:
div = rs.SlidingSpectrumDecomposition(gamma=0.5)

The pseudocode codes are:

```python
for i = 1 to n
    v[i] = a[i]
for i = 1 to n
    r[i][i] = ‖v[i]‖
    q[i] = v[i] / r[i][i]
    for j = i+1 to n
        r[i][j] = q[i][*]v[j]
        v[j] = v[j] - r[i][j]q[i]
```

The objective could be formed as:

$\max\limits_{j\in\mathcal{Y}\backslash Y}\left[r_j+\lambda\left||P_{\perp q_j}\right|| \prod\limits_{i\in Y}^{}\left||P_{\perp q_i}\right||\right]$